In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/jksrtc"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 3):  # There are 2 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('jk_bus_details.csv', index=False)

# Close the driver
driver.quit()


Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/srinagar-to-jammu: Message: 
Stacktrace:
0   chromedriver                        0x00000001006390b8 cxxbridge1$str$ptr + 1887276
1   chromedriver                        0x0000000100631794 cxxbridge1$str$ptr + 1856264
2   chromedriver                        0x000000010024082c cxxbridge1$string$len + 88524
3   chromedriver                        0x0000000100284834 cxxbridge1$string$len + 367060
4   chromedriver                        0x00000001002bc48c cxxbridge1$string$len + 595500
5   chromedriver                        0x0000000100279474 cxxbridge1$string$len + 321044
6   chromedriver                        0x000000010027a0e4 cxxbridge1$string$len + 324228
7   chromedriver                        0x0000000100600a9c cxxbridge1$str$ptr + 1656336
8   chromedriver                        0x00000001006054f8 cxxbridge1$str$ptr + 1675372
9   chromedriver                        0x00000001005e6980 cxxbridge1$str$ptr 

NameError: name 'driver' is not defined

In [2]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Jammu (j and k) to Srinagar,https://www.redbus.in/bus-tickets/jammu-to-sri...,New Pal Travels,NON A/C Seater / Sleeper (2+2),20:15,10h 00m,06:15,1.5,719,33 Seats available
1,Jammu (j and k) to Srinagar,https://www.redbus.in/bus-tickets/jammu-to-sri...,Harikesh Tour N Travels,NON A/C Seater / Sleeper (2+2),20:30,09h 30m,06:00,1.1,674,40 Seats available
2,Jammu (j and k) to Srinagar,https://www.redbus.in/bus-tickets/jammu-to-sri...,North Kashmir Tour and Travels,NON A/C Sleeper (2+2),16:30,10h 00m,02:30,1.0,INR 699,40 Seats available
3,Jammu (j and k) to Srinagar,https://www.redbus.in/bus-tickets/jammu-to-sri...,North Kashmir Tour and Travels,NON A/C Sleeper (2+2),17:15,10h 00m,03:15,1.0,INR 699,40 Seats available
4,Delhi to Srinagar,https://www.redbus.in/bus-tickets/delhi-to-sri...,DIVA BUS,AC Sleeper (2+1),15:00,17h 00m,08:00,4.3,INR 2700,33 Seats available
...,...,...,...,...,...,...,...,...,...,...
244,Jammu (j and k) to Amritsar,https://www.redbus.in/bus-tickets/jammu-to-amr...,Gajraj bus service,Bharat Benz A/C Sleeper (2+1),17:35,05h 10m,22:45,3.9,INR 1000,34 Seats available
245,Jammu (j and k) to Amritsar,https://www.redbus.in/bus-tickets/jammu-to-amr...,Gajraj bus service,Bharat Benz A/C Sleeper (2+1),18:30,05h 25m,23:55,2.2,INR 899,15 Seats available
246,Jammu (j and k) to Amritsar,https://www.redbus.in/bus-tickets/jammu-to-amr...,Vijay Tour and Travels,AC Sleeper (2+1),14:30,05h 10m,19:40,2.2,INR 800,29 Seats available
247,Jammu (j and k) to Amritsar,https://www.redbus.in/bus-tickets/jammu-to-amr...,Vijay Tour and Travels,Bharat Benz A/C Sleeper (2+1),16:30,04h 30m,21:00,0,INR 700,30 Seats available
